In [ ]:
import copy
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import display, clear_output

class Individuo:
    def __init__(self, alelos, cromosoma):
        self._alelos = alelos
        self._cromosoma = cromosoma
        self._fitness = 0

In [ ]:
class AGC:
    def __init__(self, cantidad_individuos, alelos, generaciones, p, problema, maxim = True):
        self._cantidad_individuos = cantidad_individuos
        self._alelos = alelos
        self._generaciones = generaciones
        self._p = p
        self._problema = problema
        self._maxim = maxim
        self._individuos = np.array([])

    def run(self):
        self.crearIndividuos()
        self._mejor_historico = self._individuos[0]
        mejores_historicos = [] #Para los promedios
        generacion = 1
        while generacion <= self._generaciones:
            self.evaluaIndividuos()
            self.mejor()
            hijos = np.array([])
            while len(hijos) < len(self._individuos):
                padre1 = self.ruleta()
                padre2 = self.ruleta()
                while padre1 == padre2:
                    padre2 = self.ruleta()
                h1, h2 = self.cruza(self._individuos[padre1], self._individuos[padre2])
                hijos = np.append(hijos, [h1])
                hijos = np.append(hijos, [h2])
            self.mutacion(hijos)
            self._individuos = np.copy(hijos)

            if generacion % 100 == 0:
              #Guardar los mejores historicos para promediar y graficar
              mejores_historicos.append(abs(self._mejor_historico._fitness))
              print(f'Generación: {generacion} Mejor Histórico: \ {self._mejor_historico._cromosoma} {self._mejor_historico._fitness :.5f}')
            generacion += 1
        return mejores_historicos

    def crearIndividuos(self):
        rango = (self._problema.MAX_VALUE - self._problema.MIN_VALUE)
        for i in range(self._cantidad_individuos):
            valores = np.random.random(size = self._alelos)
            cromosoma = self._problema.MIN_VALUE +  valores * rango
            individuo = Individuo(self._alelos, cromosoma)
            self._individuos = np.append(self._individuos, [individuo])

    def evaluaIndividuos(self):
        for i in self._individuos:
            i._fitness = self._problema.fitness(i._cromosoma)
            if not self._maxim:
                i._fitness *= -1

    def ruleta(self):
        f_sum = np.sum([i._fitness for i in self._individuos])
        r = np.random.randint(np.abs(f_sum + 1), dtype = np.int64)
        if f_sum < 0:
            r *= -1
        k = 0
        F = self._individuos[k]._fitness
        if f_sum < 0:
            while F > r and k < len(self._individuos)-1:
                k += 1
                F += self._individuos[k]._fitness
        else:
            while F < r and k < len(self._individuos)-1:
                k += 1
                F += self._individuos[k]._fitness
        return k

    def cruza(self, i1, i2):
        h1 = copy.deepcopy(i1)
        h2 = copy.deepcopy(i2)

        s = self._alelos - 1
        punto_cruza = np.random.randint(s) + 1
        h1._cromosoma[punto_cruza:], h2._cromosoma[punto_cruza:] = h2._cromosoma[punto_cruza:], h1._cromosoma[punto_cruza:]
        return h1, h2

    def mutacion(self, hijos):
        rango = (self._problema.MAX_VALUE - self._problema.MIN_VALUE)
        for h in hijos:
            for a in range(len(h._cromosoma)):
                if np.random.rand() < self._p:
                    h._cromosoma[a] = self._problema.MIN_VALUE + np.random.random() * rango

    def mejor(self):
        for i in self._individuos:
            if i._fitness > self._mejor_historico._fitness:
                self._mejor_historico = copy.deepcopy(i)

In [ ]:
#Funciones
class Sphere:
  MIN_VALUE = -5.12
  MAX_VALUE = 5.12
  def __init__(self):
      pass
  def fitness(self, cromosoma):
      z = 0
      for alelo in cromosoma:
          z += alelo**2
      return z

class Rosenbrock:
  MIN_VALUE = -2.048
  MAX_VALUE = 2.048
  def __init__(self):
      pass
  def fitness(self, cromosoma):
    z = 0
    x = cromosoma
    for i in range(len(cromosoma) - 1):
      z += 100 * (x[i+1] - x[i]**2) ** 2 + (x[i] - 1)**2
    return z

class Rastrigin:
  MIN_VALUE = -5.12
  MAX_VALUE = 5.12
  def __init__(self):
      pass
  def fitness(self, cromosoma, C = 10):
    z = C * len(cromosoma) #10n
    for alelo in cromosoma:
      z += alelo**2 - C * np.cos(2 * np.pi * alelo)
    return z
class Quartic:
  MIN_VALUE = -1.28
  MAX_VALUE = 1.28
  def __init__(self):
      pass
  def fitness(self, cromosoma):
    z = 0
    for alelo in cromosoma:
      z += alelo**4
    return z

In [ ]:
#Función para graficar los promedios individualmente
def graficarIndividual(arreglo):
  datos_y = arreglo
  datos_x = range(len(datos_y))

  # Graficar la línea
  plt.plot(datos_x, datos_y, marker='o')

  plt.xticks(range(0, 20))

  # Agregar etiquetas a los ejes
  plt.xlabel('Eje X')
  plt.ylabel('Eje Y')

  # Mostrar la gráfica
  plt.show()

In [ ]:
#Función para graficae los promedios en conjunto
def graficarConjunto(arregloPromedios):
    colores = ['red', 'blue', 'green']
    dimensiones = [2, 4, 8]
    datos_x = range(len(arregloPromedios[0]))  # asumimos que todos los arreglos tienen la misma longitud

    # Graficar cada arreglo en la misma gráfica con colores diferentes
    for i, arreglo in enumerate(arregloPromedios):
        color = colores[i] if colores else None
        plt.plot(datos_x, arreglo, marker='o', label=f'{dimensiones[i]} dimensiones', color=color)

    plt.xticks(range(0, 20))
    max_value = int(np.ceil(max(max(arregloPromedios))))  # Convertir el valor máximo a entero
    plt.yticks(range(0, max_value + 1, 1))
    # Agregar etiquetas a los ejes
    plt.xlabel('Eje X')
    plt.ylabel('Eje Y')

    # Mostrar la gráfica
    plt.legend()
    plt.show()

In [ ]:
def main():
  # Funciones de optimización disponibles
  optimization_functions = {
    'Sphere': Sphere(),
    'Rosenbrock': Rosenbrock(),
    'Rastrigin': Rastrigin(),
    'Quartic': Quartic()
  }

  # Arreglo para graficar los promedios en conjunto
  conjuntoPromedios = []
  # Dimensiones
  availableDimensions = [2, 4, 8]

  # Widget de selección de función
  function_selector = widgets.Dropdown(
    options=list(optimization_functions.keys()),
    value='Sphere',
    description='Función:'
  )

  # Función que se llama al hacer clic en el botón
  def on_button_click(b):
    output.clear_output()
    selected_function_name = function_selector.value
    selected_function_class = optimization_functions[selected_function_name]
    selected_function_instance = optimization_functions[selected_function_name]

    with output:
        # Ejecutar la función para cada dimensión disponible
        for dimensions in availableDimensions:
          mejores_historicosG = []
          print(f"\nFunción seleccionada: {function_selector.value}")
          print(f"Dimensiones: {dimensions}")
          for i in range(5):
              print(f"\nRun número {i+1}")
              ag = AGC(64, dimensions, 2000, 0.02, selected_function_instance, False)
              mejores_historicosG.append(ag.run())
          #Promediar
          mejores_historicosG = [sum(x) / len(x) for x in zip(*mejores_historicosG)]
          conjuntoPromedios.append(mejores_historicosG)
          print(f'\nPromedios de los mejores historicos: \n{mejores_historicosG}\n')
          #Graficar individualmente
          graficarIndividual(mejores_historicosG)
        # Graficar promedios en conjunto
        print('\nGráica con todos los promedios:\n')
        graficarConjunto(conjuntoPromedios)

  # Widget de botón
  button = widgets.Button(description="Ejecutar")
  output = widgets.Output()

  # Asignar la función al evento de clic del botón
  button.on_click(on_button_click)

  # Mostrar widgets
  display(function_selector, button, output)

In [17]:
main()

Dropdown(description='Función:', options=('Sphere', 'Rosenbrock', 'Rastrigin', 'Quartic'), value='Sphere')

Button(description='Ejecutar', style=ButtonStyle())

Output()